# Movie Ratings
This notebook uses the Guidebox API to pull in currently available movies (e.g. from HBO or Amazon Prime) and the OMDB API to find ratings on those movies. It returns the sorted results in a dataframe to help you decide what to watch. Time to quit wasting so much time trying to find a decent movie!

TODO: add in ability to filter by genre

TODO: add in Netflix as a source

In [98]:
import os
import requests
import webbrowser

import numpy as np
import pandas as pd

region = 1 # United States
api_key = os.environ.get('guidebox_api_key')

guidebox_base = 'https://api-public.guidebox.com/v1.43/%s/%s' %(region,api_key)
omdb_base = 'http://www.omdbapi.com/?'


## parameters for guidebox request

In [99]:
# limits result set from guidebox, pre-sorted by popularity
limit_1 = 1
limit_2 = 250

# change source depending on need (options = ['amazon_prime','hbo'])
sources = 'amazon_prime'
platform = 'all'


## make guidebox request

In [100]:
# construct uri based on above parameters
uri = guidebox_base + '/movies/all/%s/%s/%s/%s' %(
str(limit_1),
str(limit_2),
sources,
platform)

r = requests.get(uri)
data = r.json()
if 'Error' in data.keys():
    print 'Error: %s' %data['Error']

## parse json

In [101]:
# capture data we want to include in the dataframe
guidebox_id = [movie['id'] for movie in data['results']]
title = [movie['title'] for movie in data['results']]
imdb_id = [movie['imdb'] for movie in data['results']]

df_data = [
    guidebox_id,
    title,
    imdb_id,
]

movie_df = pd.DataFrame(df_data).transpose()
movie_df.columns=['guidebox_id','title','imdb_id']


## get review information

In [102]:
# get imdb score & rotten tomatoes for movies guidebox list

# intialize dataframe
df_cols = [
    'imdb_id',
    'imdb_rating',
    'rt_rating',
    'metascore',
    'actors',
    'rt_review',
    'awards',
]
rating_df = pd.DataFrame(columns=df_cols)

index = 0
for i in imdb_id:
    uri = omdb_base + 'i=%s&tomatoes=true&r=json' %i
    r = requests.get(uri)
    movie_data = r.json()
    df_row = pd.Series({
        'imdb_id':i,
        'imdb_rating':movie_data['imdbRating'],
        'rt_rating':movie_data['tomatoUserMeter'],
        'metascore':movie_data['Metascore'],
        'actors':movie_data['Actors'],
        'rt_review':movie_data['tomatoConsensus'],
        'awards':movie_data['Awards'],
    })
    index+=1
    rating_df.loc[index]=df_row


## combine review information with original guidebox list

In [107]:
# combine dataframes
merged = movie_df.merge(
    rating_df,
    how='inner',
    left_on='imdb_id',
    right_on='imdb_id',
)

# drop rows where rt_rating = 'N/A'
merged = merged[merged.rt_rating != 'N/A']

# sort merged dataframe by rt_score
merged.sort_values(
    by='rt_rating',
    ascending=False,
    inplace=True
)

merged.reset_index(
    inplace=True,
    drop=True,
)

# view top results
print 'Top movies available now on %s:' %sources
merged.head(20)

Top movies available now on amazon_prime:


,guidebox_id,title,imdb_id,imdb_rating,rt_rating,metascore,actors,rt_review,awards
0,79969,Pulp Fiction,tt0110912,8.9,96,94,"Tim Roth, Amanda Plummer, Laura Lovelace, John...",One of the most influential films of the 1990s...,Won 1 Oscar. Another 60 wins & 65 nominations.
1,69183,Indiana Jones and the Raiders of the Lost Ark,tt0082971,8.5,96,85,"Harrison Ford, Karen Allen, Paul Freeman, Rona...","Featuring bravura set pieces, sly humor, and w...",Won 4 Oscars. Another 30 wins & 23 nominations.
2,71723,American History X,tt0120586,8.6,96,62,"Edward Norton, Edward Furlong, Beverly D'Angel...",A compelling and provocative story led by an e...,Nominated for 1 Oscar. Another 4 wins & 13 nom...
3,47801,Amelie,tt0211915,8.4,95,69,"Audrey Tautou, Mathieu Kassovitz, Rufus, Lorel...","The feel-good Amelie is a lively, fanciful cha...",Nominated for 5 Oscars. Another 57 wins & 63 n...
4,76254,The Silence of the Lambs,tt0102926,8.6,95,84,"Jodie Foster, Lawrence A. Bonney, Kasi Lemmons...","Director Jonathan Demme's smart, taut thriller...",Won 5 Oscars. Another 48 wins & 36 nominations.
5,83711,Apocalypse Now,tt0078788,8.5,94,90,"Marlon Brando, Martin Sheen, Robert Duvall, Fr...","Francis Ford Coppola's haunting, hallucinatory...",Won 2 Oscars. Another 18 wins & 32 nominations.
6,296,Reservoir Dogs,tt0105236,8.4,94,78,"Harvey Keitel, Tim Roth, Michael Madsen, Chris...","Thrumming with intelligence and energy, Reserv...",9 wins & 15 nominations.
7,34134,Indiana Jones and the Last Crusade,tt0097576,8.3,94,65,"Harrison Ford, Sean Connery, Denholm Elliott, ...","Lighter and more comedic than its predecessor,...",Won 1 Oscar. Another 6 wins & 20 nominations.
8,202,Good Will Hunting,tt0119217,8.3,94,70,"Matt Damon, Robin Williams, Ben Affleck, Stell...","It follows a predictable narrative arc, but Go...",Won 2 Oscars. Another 20 wins & 53 nominations.
9,1854,Trainspotting,tt0117951,8.2,93,83,"Ewan McGregor, Ewen Bremner, Jonny Lee Miller,...","A brutal, often times funny, other times terri...",Nominated for 1 Oscar. Another 20 wins & 26 no...


## View Trailers (work in progress)

In [104]:
def view_trailer(guidebox_id):

    # {Base API URL} /movie/ {id} /videos/ {sources} / {limit 1} / {limit 2}
    uri = guidebox_base + '/movie/%s/videos/guidebox/1/1' %(
        guidebox_id,
    )

    # capture just the first free link
    r = requests.get(uri)
    trailer_data = r.json()
    trailer = trailer_data['results'][0]['free_web_sources'][0]['link']

    # open trailer
    webbrowser.open(trailer)

In [105]:
# try it out

view_trailer(17874)